In [1]:
import pickle
import random
import numpy as np
import torch
import os

import sys
sys.path.append('..')
from Auxiliary_Functions.generate_observation import generate_observations, split_dataset
from Auxiliary_Functions.save_progress import save_progress
from Auxiliary_Functions.matrix_normalization import normalize_max_row_norm

from Denoising_Algorithms.Projectors.DnCNN import DnCNN
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR
from Denoising_Algorithms.DL_Training.loss_functions import IntermediateScaledLoss
from Denoising_Algorithms.DL_Training.training import train_main
from Denoising_Algorithms.Nesterov_Network.Nesterov import Nesterov
from Denoising_Algorithms.DL_Training.evaluation import average_PSNR

In [2]:
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_10.pkl', 'rb') as f:
    gaussian_10 = pickle.load(f)
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_20.pkl', 'rb') as f:
    gaussian_20 = pickle.load(f)
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_30.pkl', 'rb') as f:
    gaussian_30 = pickle.load(f)
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/Gaussian_40.pkl', 'rb') as f:
    gaussian_40 = pickle.load(f)

In [3]:
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/ILSVRC2012_50x50_small.pkl', 'rb') as f:
    ILSVRC2012 = pickle.load(f)

# We only work with the first 25000 images
img_dataset = ILSVRC2012[:25000]

In [4]:
gaussian_10_normalized = normalize_max_row_norm(gaussian_10)
gaussian_20_normalized = normalize_max_row_norm(gaussian_20)
gaussian_30_normalized = normalize_max_row_norm(gaussian_30)
gaussian_40_normalized = normalize_max_row_norm(gaussian_40)

gaussian_10_observations = generate_observations(img_dataset, gaussian_10_normalized)
gaussian_20_observations = generate_observations(img_dataset, gaussian_20_normalized)
gaussian_30_observations = generate_observations(img_dataset, gaussian_30_normalized)
gaussian_40_observations = generate_observations(img_dataset, gaussian_40_normalized)

gaussian_10_train, gaussian_10_test = split_dataset(gaussian_10_observations, train_ratio = 0.9, seed = 0)
gaussian_20_train, gaussian_20_test = split_dataset(gaussian_20_observations, train_ratio = 0.9, seed = 0)
gaussian_30_train, gaussian_30_test = split_dataset(gaussian_30_observations, train_ratio = 0.9, seed = 0)
gaussian_40_train, gaussian_40_test = split_dataset(gaussian_40_observations, train_ratio = 0.9, seed = 0)

In [5]:
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_10.pkl', 'rb') as f:
    DCT_10 = pickle.load(f)
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_20.pkl', 'rb') as f:
    DCT_20 = pickle.load(f)
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_30.pkl', 'rb') as f:
    DCT_30 = pickle.load(f)
with open('/Users/wizard/Memory-Net-Inverse/Auxiliary_Functions/sensing_matrices/50 x 50 images/DCT_40.pkl', 'rb') as f:
    DCT_40 = pickle.load(f)

In [6]:
DCT_10_normalized = normalize_max_row_norm(DCT_10)
DCT_20_normalized = normalize_max_row_norm(DCT_20)
DCT_30_normalized = normalize_max_row_norm(DCT_30)
DCT_40_normalized = normalize_max_row_norm(DCT_40)

DCT_10_observations = generate_observations(img_dataset, DCT_10_normalized)
DCT_20_observations = generate_observations(img_dataset, DCT_20_normalized)
DCT_30_observations = generate_observations(img_dataset, DCT_30_normalized)
DCT_40_observations = generate_observations(img_dataset, DCT_40_normalized)

DCT_10_train, DCT_10_test = split_dataset(DCT_10_observations, train_ratio = 0.9, seed = 0)
DCT_20_train, DCT_20_test = split_dataset(DCT_20_observations, train_ratio = 0.9, seed = 0)
DCT_30_train, DCT_30_test = split_dataset(DCT_30_observations, train_ratio = 0.9, seed = 0)
DCT_40_train, DCT_40_test = split_dataset(DCT_40_observations, train_ratio = 0.9, seed = 0)

In [7]:
def calculate_average_snr(test_set: dict, sigma: float = 0.05) -> float:
    """
    Calculate average SNR across test images given noiseless measurements, excluding zero images.
    
    Parameters:
    -----------
    test_set : dict
        Dictionary containing test data with key 'X' for noiseless measurements
    sigma : float
        Standard deviation of noise to be considered
    
    Returns:
    --------
    float : Average SNR in decibels
    """
    X = test_set['X']  # These are already the noiseless measurements (Ax)
    m = X.shape[1]     # Number of measurements  
    
    signal_powers = np.sum(X**2, axis=1)
    noise_power = m * sigma**2
    
    # Filter out zero signals
    valid_signals = signal_powers > 0
    if not np.any(valid_signals):
        return None
    
    # Calculate SNR only for non-zero signals
    valid_snrs = 10 * np.log10(signal_powers[valid_signals] / noise_power)
    
    return np.mean(valid_snrs)

In [8]:
# Calculate average SNR for each DCT case
snr_10 = calculate_average_snr(DCT_10_test, sigma=0.01)
snr_20 = calculate_average_snr(DCT_20_test, sigma=0.01)
snr_30 = calculate_average_snr(DCT_30_test, sigma=0.01)
snr_40 = calculate_average_snr(DCT_40_test, sigma=0.01)

print(f"Average SNR (dB) for DCT-10: {snr_10:.2f}")
print(f"Average SNR (dB) for DCT-20: {snr_20:.2f}")
print(f"Average SNR (dB) for DCT-30: {snr_30:.2f}")
print(f"Average SNR (dB) for DCT-40: {snr_40:.2f}")

# Calculate average SNR for each DCT case
snr_10 = calculate_average_snr(DCT_10_test, sigma=0.025)
snr_20 = calculate_average_snr(DCT_20_test, sigma=0.025)
snr_30 = calculate_average_snr(DCT_30_test, sigma=0.025)
snr_40 = calculate_average_snr(DCT_40_test, sigma=0.025)

print(f"Average SNR (dB) for DCT-10: {snr_10:.2f}")
print(f"Average SNR (dB) for DCT-20: {snr_20:.2f}")
print(f"Average SNR (dB) for DCT-30: {snr_30:.2f}")
print(f"Average SNR (dB) for DCT-40: {snr_40:.2f}")

# Calculate average SNR for each DCT case
snr_10 = calculate_average_snr(DCT_10_test, sigma=0.05)
snr_20 = calculate_average_snr(DCT_20_test, sigma=0.05)
snr_30 = calculate_average_snr(DCT_30_test, sigma=0.05)
snr_40 = calculate_average_snr(DCT_40_test, sigma=0.05)

print(f"Average SNR (dB) for DCT-10: {snr_10:.2f}")
print(f"Average SNR (dB) for DCT-20: {snr_20:.2f}")
print(f"Average SNR (dB) for DCT-30: {snr_30:.2f}")
print(f"Average SNR (dB) for DCT-40: {snr_40:.2f}")

Average SNR (dB) for DCT-10: 42.61
Average SNR (dB) for DCT-20: 39.61
Average SNR (dB) for DCT-30: 37.87
Average SNR (dB) for DCT-40: 36.63
Average SNR (dB) for DCT-10: 34.65
Average SNR (dB) for DCT-20: 31.65
Average SNR (dB) for DCT-30: 29.91
Average SNR (dB) for DCT-40: 28.67
Average SNR (dB) for DCT-10: 28.63
Average SNR (dB) for DCT-20: 25.63
Average SNR (dB) for DCT-30: 23.89
Average SNR (dB) for DCT-40: 22.65


In [9]:
# Calculate average SNR for each DCT case
snr_10 = calculate_average_snr(gaussian_10_test, sigma=0.01)
snr_20 = calculate_average_snr(gaussian_20_test, sigma=0.01)
snr_30 = calculate_average_snr(gaussian_30_test, sigma=0.01)
snr_40 = calculate_average_snr(gaussian_40_test, sigma=0.01)

print(f"Average SNR (dB) for Gauss-10: {snr_10:.2f}")
print(f"Average SNR (dB) for Gauss-20: {snr_20:.2f}")
print(f"Average SNR (dB) for Gauss-30: {snr_30:.2f}")
print(f"Average SNR (dB) for Gauss-40: {snr_40:.2f}")

# Calculate average SNR for each DCT case
snr_10 = calculate_average_snr(gaussian_10_test, sigma=0.025)
snr_20 = calculate_average_snr(gaussian_20_test, sigma=0.025)
snr_30 = calculate_average_snr(gaussian_30_test, sigma=0.025)
snr_40 = calculate_average_snr(gaussian_40_test, sigma=0.025)

print(f"Average SNR (dB) for Gauss-10: {snr_10:.2f}")
print(f"Average SNR (dB) for Gauss-20: {snr_20:.2f}")
print(f"Average SNR (dB) for Gauss-30: {snr_30:.2f}")
print(f"Average SNR (dB) for Gauss-40: {snr_40:.2f}")

# Calculate average SNR for each DCT case
snr_10 = calculate_average_snr(gaussian_10_test, sigma=0.05)
snr_20 = calculate_average_snr(gaussian_20_test, sigma=0.05)
snr_30 = calculate_average_snr(gaussian_30_test, sigma=0.05)
snr_40 = calculate_average_snr(gaussian_40_test, sigma=0.05)

print(f"Average SNR (dB) for Gauss-10: {snr_10:.2f}")
print(f"Average SNR (dB) for Gauss-20: {snr_20:.2f}")
print(f"Average SNR (dB) for Gauss-30: {snr_30:.2f}")
print(f"Average SNR (dB) for Gauss-40: {snr_40:.2f}")

Average SNR (dB) for Gauss-10: 32.19
Average SNR (dB) for Gauss-20: 32.55
Average SNR (dB) for Gauss-30: 32.57
Average SNR (dB) for Gauss-40: 32.49
Average SNR (dB) for Gauss-10: 24.23
Average SNR (dB) for Gauss-20: 24.59
Average SNR (dB) for Gauss-30: 24.62
Average SNR (dB) for Gauss-40: 24.53
Average SNR (dB) for Gauss-10: 18.21
Average SNR (dB) for Gauss-20: 18.57
Average SNR (dB) for Gauss-30: 18.59
Average SNR (dB) for Gauss-40: 18.51
